### Mounting Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/Storma/machine-learning-challenge

### Install Requirements

In [ ]:
# !pip3 install -r requirements.txt
# !conda install pytorch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 cudatoolkit=11.3 -c pytorch

### Check Nvidia and Cuda

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.__version__  # '1.10.0+cu113'

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.get_device_properties(0).name

### Clone YOLOv5

In [ ]:
import os

HOME = os.getcwd()
print(HOME)

In [ ]:
%cd {HOME}

# clone YOLOv5
# !git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git pull
%pip install -r requirements.txt
!pip uninstall comet_ml  -y

import utils
display = utils.notebook_init()

In [ ]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

print(
    f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})"
)

### Global Variables

In [ ]:
yolo_version = "5"

coco_dataset_dir = os.path.join(HOME, "data/coco")
yolo_dataset_dir = os.path.join(HOME, "data/yolo")
experiments_folder = os.path.join(HOME, "yolov{}/runs/train/".format(yolo_version))
weights_folder = os.path.join(HOME, "yolov{}/runs/detect/".format(yolo_version))
yolo_weights_folder = os.path.join(HOME, "weights".format(yolo_version))
pretrained_weights_path = os.path.join(yolo_weights_folder, "yolov5l.pt")

# set up environment
os.environ["DATASET_DIRECTORY"] = yolo_dataset_dir

### Download YOLO pretained weights

In [ ]:
%cd {HOME}

In [ ]:
!mkdir -p {yolo_weights_folder}
%cd {yolo_weights_folder}

# !wget https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5x6.pt
# !wget https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5x.pt
# !wget https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5l.pt
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

%cd {HOME}

### Train Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `DATASET_DIRECTORY`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training
- **hyp:** determine hyperparameters yaml file


##### Clean GPU memory before training

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

##### Start training

In [ ]:
!python yolov{yolo_version}/train.py \
    --batch 2 \
    --epochs 20 \
    --data {yolo_dataset_dir}/custom.yaml  \
    --weights {pretrained_weights_path} \
    --project {experiments_folder} \
    --device 0

In [ ]:
!ls {experiments_folder}

##### Start evalutation

In [ ]:
!python yolov{yolo_version}/val.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.pt \
    --data {yolo_dataset_dir}/custom.yaml \
    --img 640 \
    --verbose \
    --device cpu

### Run inference with the trained YOLOv5 model

#### Run inference on images

In [ ]:
!python yolov{yolo_version}/detect.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.pt \
    --img 640 \
    --conf 0.5 \
    --source {yolo_dataset_dir}/test/images \
    --project {weights_folder}

#### Run inference on videos

In [ ]:
!python yolov{yolo_version}/detect.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.pt \
    --img 640 \
    --conf 0.5 \
    --source {coco_dataset_dir}/images/test/test.mp4 \
    --project {weights_folder}

In [ ]:
!ls {weights_folder}

### Prediction Function For Inference

In [ ]:
import torch
import cv2
import os
import glob
from IPython.display import Image, display

def model_predict(image, model, size=416, output_folder="runs/detect/exp/", show_img=False):

    """
    image: can be: file, Path, PIL, OpenCV, numpy, list
    model: weights file of trained torch model
    size: size of the image
    out_img_path: where to save the result image
    """

    # Inference
    results = model(image, size=size)
    
    # Show Results
    if show_img:
        results.show()
        
    # Save Results
    if output_folder:
        results.save(save_dir=output_folder, exist_ok=True)  
    
    # results.crop(), results.pandas(), etc.
    # print(results.xyxy[0])  # im predictions (tensor)
    # print(results.pandas().xyxy[0])  # im predictions (pandas)
    # print(results.pandas().xyxy[0].value_counts('name'))  # class counts (pandas)


Get model path of the last/best experiment

In [ ]:
models_paths = glob.glob(os.path.join(HOME, experiments_folder, "*"))
models_paths = sorted(models_paths, key=os.path.getmtime)
model_path = os.path.join(models_paths[-1], "weights/best.pt")
print("Using model weights file:\n{}\n".format(model_path))

Load inference model

In [ ]:
model = torch.hub.load(f"{HOME}/yolov{yolo_version}", "custom", path=model_path, source="local")
if torch.cuda.is_available():
    model.cuda()

print("Model is loaded!")

Create output folder

In [ ]:
from yolov5.utils.general import increment_path
from pathlib import Path


test_imgs_folder = os.path.join(HOME, yolo_dataset_dir, "test/images")
print(test_imgs_folder)

output_folder = os.path.join(HOME, 'yolov{}/runs/detect/exp/'.format(yolo_version))
output_folder = increment_path(Path(output_folder), exist_ok=False)
print(output_folder)

if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

Run inference

In [ ]:
from PIL import Image


image_paths = [image_path for image_path in glob.glob(test_imgs_folder + "/*")][1000:1050]

for image_path in image_paths:
    print(image_path)
    img_file_name = os.path.basename(image_path)
    model_predict(
        image_path, model=model, output_folder=output_folder, show_img = True
    )


### YOLOv5 with StrongSORT Inference

In [ ]:
import sys
sys.path.append("yolov5")

from yolov5.utils.general import increment_path
from pathlib import Path
import glob
import os

yolo_version = "5"

HOME = os.getcwd()
coco_dataset_dir = os.path.join(HOME, "data/coco/")
video_name = "test"
input_video = os.path.join(coco_dataset_dir, "images/test/{}.mp4".format(video_name))
print("[INFO] input_video:", input_video)

output_folder = os.path.join(HOME, "yolov{}/runs/track/exp/".format(yolo_version))
output_folder = increment_path(Path(output_folder), exist_ok=False)
print("[INFO] output_folder:", output_folder)

if not os.path.isdir(output_folder):
    os.mkdir(output_folder)
    
experiments_folder = os.path.join(HOME, "yolov{}/runs/train/".format(yolo_version))
models_paths = glob.glob(os.path.join(HOME, experiments_folder, "*"))
models_paths = sorted(models_paths, key=os.path.getmtime)
yolo_model_path = os.path.join(models_paths[-1], "weights/best.pt")
print("[INFO] yolo_model_path:{}".format(yolo_model_path))

strong_sort_mode_path =  os.path.join(HOME, "weights/osnet_x0_25_msmt17.pt")
print("[INFO] strong_sort_mode_path:{}".format(strong_sort_mode_path))

In [ ]:
!python3 strongsort_yolov5_track.py \
    --source {input_video}   \
    --yolo-weights {yolo_model_path}  \
    --strong-sort-weights {strong_sort_mode_path} \
    --project {output_folder}  \
    --name {video_name}  \
    --device 0 \
    --save-txt \
    --count \
    --save-conf \
    --save-vid \
    --exist-ok
#  --show-vid

### Inference optimization (ONNX)

##### Export a Trained YOLOv5 Model to ONNX format

In [ ]:
input_width = 640 
input_height = 640

!python3 yolov5/export.py \
    --weights ./yolov5/runs/train/exp2/weights/best.pt \
    --img {input_height} {input_width} \
    --batch 1 \
    --include "onnx" \
    --simplify \
    --device 0

##### ONNX Runtime Evaluation

In [ ]:
!python yolov{yolo_version}/val.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.onnx \
    --data {yolo_dataset_dir}/custom.yaml \
    --img 640 \
    --verbose \
    --batch-size 8 \
    --device 0

##### ONNX Runtime Model Inference

In [ ]:
!python yolov{yolo_version}/detect.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.onnx \
    --img 640 \
    --conf 0.5 \
    --source {coco_dataset_dir}/images/test/test.mp4 \
    --project {weights_folder}

### Inference optimization (TensorRT)

##### Export a Trained YOLOv5 Model to TensorRT format

In [ ]:
input_width = 640 
input_height = 640

!python3 yolov5/export.py \
    --weights ./yolov5/runs/train/exp2/weights/best.pt \
    --img {input_height} {input_width} \
    --batch 1 \
    --include "engine" \
    --dynamic \
    --device 0

##### TensorRT Evaluation

In [ ]:
!python yolov{yolo_version}/val.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.engine \
    --data {yolo_dataset_dir}/custom.yaml \
    --img 640 \
    --verbose \
    --batch-size 8 \
    --device 0

##### TensorRT Model Inference

In [ ]:
!python yolov{yolo_version}/detect.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.engine \
    --img 640 \
    --conf 0.5 \
    --data {yolo_dataset_dir}/custom.yaml \
    --source {coco_dataset_dir}/images/test/test.mp4 \
    --project {weights_folder}

### Run YOLOv5 benchmarks (speed and accuracy) for all supported export formats.

In [ ]:
!python yolov{yolo_version}/benchmarks.py \
    --weights yolov{yolo_version}/runs/train/exp2/weights/best.pt \
    --data {yolo_dataset_dir}/custom.yaml \
    --imgsz 640 \
    --device 0